# Q15 특정 거리의 도시 찾기

In [ ]:
import sys
from collections import deque

def bfs(node):
    queue = deque([node])
    depth[node] = 0

    while queue:
        curr = queue.popleft()
        for next in graph[curr]:
            if depth[next] == -1:
                queue.append(next)
                depth[next] = depth[curr] + 1

n, m, k, x = map(int, input().split())
graph = [[] for _ in range(n + 1)]
depth = [-1] * (n + 1)

for _ in range(m):
    a, b = map(int, sys.stdin.readline().rstrip().split())
    graph[a].append(b)

bfs(x)

if k not in depth:
    print(-1)
else:
    for idx in range(1, n + 1):
        if depth[idx] == k:
            print(idx)



4 4 1 1
1 2
1 3
2 3
2 4
2
3


* **모든 도로의 거리는 1**이라는 조건 덕분에, BFS를 이용하여 간단히 풀 수 있는 문제.
* 노드 개수 $N$은 최대 300,000개, 간선 개수 $M$은 최대 1,000,000개이므로, $O(N+M)$으로 동작하는 소스코드를 작성할 수 있음.
  * 인접리스트 사용 시, $N$은 노드를 큐에 넣고 꺼내는 작업. $M$은 간선을 검사하는 작업.
* 시작점부터 BFS를 수행하여 모든 도시까지 최단 거리를 계산한 뒤, 최단 거리가 K인 경우 출력.

In [ ]:
# 모범답안
import sys
from collections import deque

def bfs(node):
    queue = deque([node])
    depth[node] = 0

    while queue:
        curr = queue.popleft()
        for next in graph[curr]:
            if depth[next] == -1:
                queue.append(next)
                depth[next] = depth[curr] + 1

n, m, k, x = map(int, input().split())
graph = [[] for _ in range(n + 1)]
depth = [-1] * (n + 1)

for _ in range(m):
    a, b = map(int, sys.stdin.readline().rstrip().split())
    graph[a].append(b)

bfs(x)

check = False

for idx in range(1, n + 1):
    if depth[idx] == k:
        print(idx)
        check = True

if not check:
    print(-1)


# Q16 연구소

In [ ]:
from collections import deque
from itertools import combinations
import copy
import sys

# 바이러스 확산을 bfs로 구현
def bfs(wall_grid, x, y):
    dx = [-1, 0, 1, 0]
    dy = [0, -1, 0, 1]
    queue = deque([(x, y)])

    while queue:
        cx, cy = queue.popleft()
        for idx in range(4):
            nx, ny = cx + dx[idx], cy + dy[idx]
            if 0 <= nx < n and 0 <= ny < m and wall_grid[nx][ny] == 0:
                queue.append((nx, ny))
                wall_grid[nx][ny] = 2

# input
n, m = map(int, input().split())
grid = []

place_walls = [] # 벽을 세울 수 있는 공간, 즉 0

for row in range(n):
    row_input = list(map(int, sys.stdin.readline().rstrip().split()))
    grid.append(row_input)
    for col in range(m):
        if row_input[col] == 0:
            place_walls.append((row, col))

answer = 0

for cmb in combinations(place_walls, 3):
    wall_grid = copy.deepcopy(grid)
    safe = 0

    # 벽 세우기
    for x, y in cmb:
        wall_grid[x][y] = 1

    # 전염하기
    for x in range(n):
        for y in range(m):
            if grid[x][y] == 2:
                bfs(wall_grid, x, y)

    # 안전한 곳은?
    safe = sum(row.count(0) for row in wall_grid)
    answer = max(safe, answer)

print(answer)

4 6
0 0 0 0 0 0
1 0 0 0 0 2
1 1 1 0 0 2
0 0 0 0 0 2
9


* 모든 경우의 수를 고려해도 됨. 전체 맵의 크기가 8x8이므로 $64\choose3$인데, 이는 100,000보다 작은 수임.
* 벽의 개수가 3개가 되는 모든 조합을 찾기 --> 조합 라이브러리 혹은 DFS나 BFS를 사용할 수 있다.
* 안전 영역의 크기를 계산하기 --> DFS나 BFS를 사용할 수 있다.

In [ ]:
# 모범답안은 DFS를 사용했다

# input
n, m = map(int, input().split())
before_grid = [] # 초기의 grid
after_grid = [[0] * m for _ in range(n)] # 벽 설치 후 grid

for _ in range(n):
    before_grid.append(list(map(int, input().split())))

dx = [-1, 0, 1, 0]
dy = [0, 1, 0, -1]

answer = 0

# DFS를 통해 바이러스 퍼뜨리기
def virus(x, y):
    for i in range(4):
        nx = x + dx[i]
        ny = y + dy[i]

        if 0 <= nx < n and 0 <= ny < m and after_grid[nx][ny] == 0:
            after_grid[nx][ny] = 2
            virus(nx, ny)

# 안전 영역의 크기
def get_score():
    score = 0
    for i in range(n):
        for j in range(m):
            if after_grid[i][j] == 0:
                score += 1
    return score

# DFS를 이용해 울타리를 설치하면서 매번 안전 영역의 크기 계산
def dfs(count):
    global answer

    # 울타리 3개 설치
    if count == 3:
        for i in range(n):
            for j in range(m):
                after_grid[i][j] = before_grid[i][j] # 그대로 지정하면 복사 아니라 할당이 됨에 주이ㅡ

        for i in range(n):
            for j in range(m):
                if after_grid[i][j] == 2:
                    virus(i, j)

        answer = max(answer, get_score())
        return

    for i in range(n):
        for j in range(m):
            if before_grid[i][j] == 0:
                before_grid[i][j] = 1
                count += 1
                dfs(count)
                before_grid[i][j] = 0
                count -= 1

dfs(0)
print(answer)

4 6
0 0 0 0 0 0
1 0 0 0 0 2
1 1 1 0 0 2
0 0 0 0 0 2
0


# Q17 경쟁적 전염

In [ ]:
# input
n, k = map(int, input().split())
grid = []

# 1초 전염 시작 위치 기록
order = []
for row in range(n):
    grid.append(list(map(int, input().split())))
    for col in range(n):
        if grid[row][col] >= 1:
            order.append((grid[row][col], row, col))
order.sort()

s, x, y = map(int, input().split())

# 전염 시뮬레이션 - 매초 전염 시작 위치 갱신
def bfs(order):
    dx = [-1, 0, 1, 0]
    dy = [0, -1, 0, 1]
    next_order = []

    while order:
        virusno, cx, cy = order.pop(0)
        for idx in range(4):
            nx, ny = cx + dx[idx], cy + dy[idx]
            if 0 <= nx < n and 0 <= ny < n and grid[nx][ny] == 0:
                grid[nx][ny] = virusno
                next_order.append((virusno, nx, ny))
    next_order.sort()

    return next_order

# 매초마다 전염
for _ in range(s):
    order = bfs(order)

# 정답 출력
print(grid[x-1][y-1])


3 3
1 0 2
0 0 0
3 0 0
1 2 2
0


* 엄밀히 말하면 일반적인 BFS가 아닌 게, 무조건 번호가 낮은 바이러스부터 증식해야 한다는 조건이 있음
* 나는 그냥 리스트를 썼지만, 큐를 쓸 거면 낮은 바이러스의 번호부터 삽입해야 함


In [ ]:
# 모범답안
from collections import deque

# input
n, k = map(int, input().split())
grid = []
virus = []

for row in range(n):
    grid.append(list(map(int, input().split())))
    for col in range(n):
        if grid[row][col] > 0:
            # 바이러스 종류, 시간, 위치 행 및 열
            virus.append((grid[row][col], 0, row, col))

# queue는 sort할 수 없다. list를 sort한 뒤 queue에 넣어 주자.
virus.sort()
queue = deque(virus)

s, x, y = map(int, input().split())
dx = [-1, 0, 1, 0]
dy = [0, 1, 0, -1]

# BFS의 특성상 매 초마다 낮은 숫자의 바이러스가 먼저 올 수밖에 없다. 즉 다시 정렬을 할 필요는 없음.
while queue:
    virusno, cs, cx, cy = queue.popleft()
    if cs == s:
        break
    for idx in range(4):
        nx, ny = cx + dx[idx], cy + dy[idx]
        if 0 <= nx < n and 0 <= ny < n and grid[nx][ny] == 0:
            grid[nx][ny] = virusno
            queue.append((virusno, cs + 1, nx, ny))

# 정답 출력
print(grid[x-1][y-1])


3 3
1 0 2
0 0 0
3 0 0
1 2 2
0


# Q18 괄호 변환

In [ ]:
def check_right(p):
    stack = []

    for lett in p:
        if lett == "(":
            stack.append("(")
        else:
            if stack:
                stack.pop()
            else:
                return False
    else:
        return not stack

def check_balanced(p):
    return p.count("(") == len(p) // 2


def solution(p):
    if check_right(p) or p == "":
        answer = p
    else:
        for idx in range(2, len(p) + 2, 2):
            u = p[:idx]
            v = p[idx:]
            if check_balanced(u) and check_balanced(v):
                break
        if check_right(u):
            answer = u + solution(v)
        else:
            answer = "(" + solution(v) + ")"
            for lett in u[1:-1]:
                if lett == "(":
                    answer += ")"
                else:
                    answer += "("
    return answer

* 구현, 재귀함수 문제에 가깝다.
* 균형잡힌 괄호 문자열의 인덱스를 반환하는 함수, 올바른 괄호 문자열인지 판단하는 함수를 별도로 구현하는 게 좋다.

In [ ]:
# 올바른 문자열인지 판단 - 사실 스택을 안 써도 됨
def check_right(p):
    count = 0
    for lett in p:
        if lett == "(":
            count += 1
        else:
            if count == 0:
                return False
            count -= 1
    return count == 0

# 균형잡힌 괄호 문자열의 인덱스
def balanced_index(p):
    count = 0
    for i in range(len(p)):
        if p[i] == "(":
            count += 1
        else:
            count -= 1
        if count == 0:
            return i

def solution(p):
    if check_right(p) or p == "":
        answer = p
    else:
        idx = balanced_index(p)
        u = p[:idx + 1]
        v = p[idx + 1:]
        if check_right(u):
            answer = u + solution(v)
        else:
            answer = "(" + solution(v) + ")"
            for lett in u[1:-1]:
                if lett == "(":
                    answer += ")"
                else:
                    answer += "("
    return answer

# Q19 연산자 끼워 넣기

In [ ]:
n = int(input())
nums = list(map(int, input().split()))
plus, minus, times, div = map(int, input().split())
results = []

def calc(curr, idx, plus, minus, times, div):
    if idx >= n:
        results.append(curr)
    else:
        next = nums[idx]

        if plus > 0:
            calc(curr + next, idx + 1, plus - 1, minus, times, div)

        if minus > 0:
            calc(curr - next, idx + 1, plus, minus - 1, times, div)

        if times > 0:
            calc(curr * next, idx + 1, plus, minus, times - 1, div)

        if div > 0:
            if curr < 0:
                result = -1 * ((-1 * curr) // next)
            else:
                result = curr // next
            calc(result, idx + 1, plus, minus, times, div - 1)

calc(nums[0], 1, plus, minus, times, div)

print(max(results))
print(min(results))

6
1 2 3 4 5 6
2 1 1 1
54
-24


In [ ]:
n = int(input())
nums = list(map(int, input().split()))
pmtd = list(map(int, input().split()))
results = []

def calc(curr, idx, pmtd):
    if idx >= n:
        results.append(curr)
    else:
        next = nums[idx]

        for c in range(4):
            if pmtd[c] > 0:
                new_pmtd = pmtd.copy()
                new_pmtd[c] -= 1

                if c == 0:
                    calc(curr + next, idx + 1, new_pmtd)
                elif c == 1:
                    calc(curr - next, idx + 1, new_pmtd)
                elif c == 2:
                    calc(curr * next, idx + 1, new_pmtd)
                elif c == 3:
                    if curr < 0:
                        result = -1 * ((-1 * curr) // next)
                    else:
                        result = curr // next
                    calc(result, idx + 1, new_pmtd)

calc(nums[0], 1, pmtd)

print(max(results))
print(min(results))

6
1 2 3 4 5 6
2 1 1 1
54
-24


In [ ]:
-13 // 7  # (13 // 7 = 1, -1)

-2

* 이게 왜 DFS BFS 문제인거지??? 재귀라도 써야 하나???
* 그러네! $2 \leq N \leq 11$이기 때문에 재귀가 얼마 안 이루어져서 충분히 감당 가능하네!
* 남은 사칙연산 수를 리스트로 구현하다가 할당, copy 이루어지는게 복잡해서 걍 각 변수 따로 할당하는 걸로 바꿨다. 근데 리스트로 구현할 방법은 정녕 없나? -> 매 재귀함수 호출 전에 새로 만들어줘야 함.
* 이게 싫으면 `itertools`의 `product` 클래스로 계산해도 됨.
* 오타 좀 내지 마요!!


In [ ]:
# 모범답안
n = int(input())
nums = list(map(int, input().split()))
plus, minus, times, div = map(int, input().split())
min_value = 1e9
max_value = -1e9

def calc(curr, idx):
    global min_value, max_value, plus, minus, times, div
    if idx >= n:
        min_value = min(min_value, curr)
        max_value = max(max_value, curr)
    else:
        next = nums[idx]
        if plus > 0:
            plus -= 1
            calc(curr + next, idx + 1)
            plus += 1

        if minus > 0:
            minus -= 1
            calc(curr - next, idx + 1)
            minus += 1

        if times > 0:
            times -= 1
            calc(curr * next, idx + 1)
            times += 1

        if div > 0:
            div -= 1
            calc(int(curr / next), idx + 1)
            div += 1

calc(nums[0], 1)
print(max_value)
print(min_value)

6
1 2 3 4 5 6
2 1 1 1
54
-24


# Q20 감시 피하기

In [ ]:
from itertools import combinations
import copy

n = int(input())
grid = []
blank = [] # 빈칸, 즉 장애물 설치가 가능한 곳
teachers = [] # 선생님들이 있는 곳

for r in range(n):
    grid.append(input().split())
    for c in range(n):
        if grid[r][c] == "X":
            blank.append((r, c))
        elif grid[r][c] == "T":
            teachers.append((r, c))

# 선생님의 감시
def spy_teach(tx, ty):
    # 학생이 들킨 경우 False, 들키지 않은 경우 True
    dx = [-1, 0, 1, 0]
    dy = [0, -1, 0, 1]

    for idx in range(4):
        cx, cy = tx, ty
        while True:
            nx, ny = cx + dx[idx], cy + dy[idx]
            if 0 > nx or n <= nx or 0 > ny or n <= ny:
                break
            elif new_grid[nx][ny] == "O":
                break
            elif new_grid[nx][ny] == "S":
                # 이런, 들키고 말았구나
                return False
            else:
                cx, cy = nx, ny
    # 학생이 들키지 않음
    return True

# 모든 선생님의 감시
# 1명의 선생님한테라도 학생이 들킨 경우: False
# 모두에게 들키지 않은 경우: TRUE
def spy_teachers(teachers):
    for tx, ty in teachers:
        if not spy_teach(tx, ty):
            return False
    return True

# 장애물을 3개 설치할 수 있는 모든 경우의 수 고려
for cmb_3 in combinations(blank, 3):
    new_grid = copy.deepcopy(grid)

    for ox, oy in cmb_3:
        new_grid[ox][oy] = "O"

    if spy_teachers(teachers):
        print("YES")
        break

else:
    print("NO")



5
X S X X T
T X S X X
X X X X X
X T X X X
X X T X X
YES


* 이건 BFS/DFS 문제인가, 조건문 문제인가...
* BFS/DFS로도 가능한 위치 파악이 가능하지만, 모범답안에선 나럼 그냥 조합 라이브러리 사용.
* 방향 역시 함수에 넣었으면 어땠을까?

In [ ]:
from itertools import combinations
import copy

n = int(input())
grid = []
blank = [] # 빈칸, 즉 장애물 설치가 가능한 곳
teachers = [] # 선생님들이 있는 곳

for r in range(n):
    grid.append(input().split())
    for c in range(n):
        if grid[r][c] == "X":
            blank.append((r, c))
        elif grid[r][c] == "T":
            teachers.append((r, c))

# 선생님의 감시 - 방향별로 구현
def spy_teach(tx, ty, dir_idx):
    dx = [-1, 0, 1, 0]
    dy = [0, -1, 0, 1]

    # 학생이 들킨 경우 True, 들키지 않은 경우 False
    while 0 <= tx < n and 0 <= ty < n:
        if new_grid[tx][ty] == "S":
            # 학생 들킴
            return True
        if new_grid[tx][ty] == "O":
            # 장애물 침
            return False
        tx += dx[dir_idx]
        ty += dy[dir_idx]
    # 벽 침
    return False

# 모든 선생님의 감시 - 장애물 설치 후
# 학생이 들킨 경우 True, 들키지 않은 경우 False
def process():
    for tx, ty in teachers:
        for dir_idx in range(4):
            if spy_teach(tx, ty, dir_idx):
                return True

    return False


# 장애물을 3개 설치할 수 있는 모든 경우의 수 고려
for cmb_3 in combinations(blank, 3):
    new_grid = copy.deepcopy(grid)

    for ox, oy in cmb_3:
        new_grid[ox][oy] = "O"

    if not process():
        print("YES")
        break

else:
    print("NO")



5
X S X X T
T X S X X
X X X X X
X T X X X
X X T X X
YES


# Q21 인구 이동

In [52]:
from collections import deque
import sys

n, l, r = map(int, input().split())
grid = []
answer = 0 # 몇 번 인구이동?

for _ in range(n):
    grid.append(list(map(int, sys.stdin.readline().rstrip().split())))

# 연합 찾기
def bfs(x, y):
    queue = deque([(x, y)])
    visited[x][y] = True
    union = [(x, y)]
    dx = [-1, 0, 1, 0]
    dy = [0, -1, 0, 1]

    while queue:
        cx, cy = queue.popleft()

        for i in range(4):
            nx, ny = cx + dx[i], cy + dy[i]
            if 0 <= nx < n and 0 <= ny < n and not visited[nx][ny]:
                    if l <= abs(grid[cx][cy] - grid[nx][ny]) <= r:
                        queue.append((nx, ny))
                        union.append((nx, ny))
                        visited[nx][ny] = True

    return union


# (연합개수 확인) -> (인구이동)
while True:
    visited = [[False] * n for _ in range(n)]
    unions = []

    # BFS 통해 연합 확인
    for x in range(n):
        for y in range(n):
            if not visited[x][y]:
                union = bfs(x, y)
                if len(union) >= 2:
                    unions.append(union)

    if not unions:
        # 인구 이동 불가
        print(answer)
        break

    else:
        # 인구 이동 가능
        answer += 1
        for union in unions:
            union_sum = 0
            for ux, uy in union:
                union_sum += grid[ux][uy]
            union_mean = union_sum // len(union)
            for ux, uy in union:
                grid[ux][uy] = union_mean


2 20 50
50 30
20 40
1


* 그래프를 꼭 만들어야 하나? 그럴 필요 없었네 ㅠㅠ
* 모든 나라의 위치에서 DFS 혹은 BFS를 수행하여 인접한 나라의 인구수를 확인한 뒤에, 가능하다면 국경선을 열고 인구 이동 처리하기.

In [60]:
# 모범답안
from collections import deque

n, l, r = map(int, input().split())
grid = []
answer = 0 # 몇 번 인구이동?

for _ in range(n):
    grid.append(list(map(int, input().split())))

# 연합 찾기 - 현재 연합 번호를 union_id에 할당
def bfs(x, y, union_id):
    # 연결된 나라 정보의 리스트
    united = [(x, y)]
    queue = deque([(x, y)])
    visited[x][y] = True
    union[x][y] = union_id # 현재 연합의 번호 할당
    union_pop = grid[x][y] # 현재 연합의 전체 인구 수
    count = 1 # 현재 연합의 국가 수
    dx = [-1, 0, 1, 0]
    dy = [0, -1, 0, 1]

    while queue:
        x, y = queue.popleft()

        for i in range(4):
            nx, ny = x + dx[i], y + dy[i]
            if 0 <= nx < n and 0 <= ny < n and union[nx][ny] == -1:
                    if l <= abs(grid[x][y] - grid[nx][ny]) <= r:
                        queue.append((nx, ny))
                        union[nx][ny] = union_id
                        union_pop += grid[nx][ny]
                        count += 1
                        united.append((nx, ny))

    for i, j in united:
        grid[i][j] = union_pop // count
    return count

total_count = 0 # 총 인구이동 횟수


# (연합개수 확인) -> (인구이동)
while True:
    union = [[-1] * n for _ in range(n)]
    union_id = 0

    # BFS 통해 연합 확인
    for x in range(n):
        for y in range(n):
            if union[x][y] == -1:
                bfs(x, y, union_id)
                union_id += 1

    if union_id == n * n:
        # union이 없는 경우. 이 경우 모든 x, yunion[x][y] == -1이므로 break 할 수 있다.
        break
    total_count += 1

print(total_count)

3 5 10
10 15 20
20 30 25
40 22 10
2
